# Create a sequence generator with RNN

In this notebook, we develog a Recurrent Neural Network (RNN) to create a Galician language sequence generator. In this project, we use text from the Galician politician Beiras. 

In http://52.24.208.72:8888/notebooks/beiras-rnn/Beiras-rnn.ipynb we choose  GRU as our best network, here we try some different hyperparameters:

Move the learning rate: With 0.001 work perfect, when we reduce the net does not learn.

Change to Adam Optimizer: We get similar results than using RSMprop

Reduce batch size from 500 to 32.- This increase the time and I could not finish the learning process.

It normally takes 10 hours to train the models in a g2.2xlarge AWS machine.

This work is based in 
https://github.com/udacity/aind2-rnn/blob/master/RNN_project.ipynb


## Load the data
First we load the data and preprocess:
* Lower
* remove lines with http links
* remove symbols: '[ºªàâäçèêïìôöü&%@•…«»”“*/!"(),.:;_¿¡¿‘’´\[\]\']'

In [1]:
import sys
sys.path.insert(0, '../aux/')
import numpy as np
import timeit
from beiras_aux import load_text,predict_next_chars,print_predicctions


In [2]:
window_size = 100
step_size = 1
X,y,chars,chars_to_indices,indices_to_chars,text_clean=load_text('../data/Beiras.txt',window_size,step_size);

* X .- Array shape (sentences, window_size, num_chars) .- Input for training.
* y .- Array shape (sentences, num_chars) .- Output for training.
* chars . -Array with chars we have in the clean text
* chars_to_indices,indices_to_chars .- dictionaries to convert fron number to char and char to index
* text_clean .- All the text clean.

## Test we have a GPU
I used a g2.2xlarge EC2 machine. Without a GPU this is too slow.

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5599031778349300983, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 3868721152
 locality {
   bus_id: 1
 }
 incarnation: 6854826919984061313
 physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"]

## Auxilar function to write a log when learning.

In [4]:
def add_log(log_string):
    """
    Add log_string to file "log-hyper.txt
    """
    with open("log-hyper.txt", "a") as myfile:
        myfile.write(log_string + "\n")  
        

In [13]:
class LossHistory(keras.callbacks.Callback):
    """
    Callback to be called after every epoch
    """
    def on_epoch_end(self, batch, logs={}):
        add_log(str(logs))
history = LossHistory()


In [ ]:
#to measure the time
import timeit

## Define network

In [5]:
### necessary functions from the keras library
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random
from keras.layers import Dense, Activation,GRU
def create_gru_model(chars):
    num_chars = len(chars)
    model= Sequential()
    # 1 Layer .- LSTM layer 1 should be an LSTM module with 200 hidden units
    model.add(GRU(200,input_shape = (window_size,num_chars),return_sequences=True))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(GRU(200))
    model.add(Dense(num_chars,activation='softmax'))
    # initialize optimizer
    optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    # compile model --> make sure initialized optimizer and callbacks - as defined above - are used
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

Using TensorFlow backend.


loss='categorical_crossentropy'
RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

sobre 1225 s train

In [6]:
epoch=30

## lr 0.001 RMSprot batch 500

In [ ]:
add_log("\n\nlr 0.001 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('../model_weights/best_beiras_gru_textdata_weights-lr001.hdf5')


/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
 547000/1174280 [============>.................] - ETA: 645s - loss: 2.0926 

In [13]:
model=create_gru_model(chars)
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights.hdf5',
                   chars_to_indices,indices_to_chars,text_clean,window_size)


pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran.... un contrasentido arestora e a construción de anos de autonomía galega non é unha concepción do seu 
poema de rosalía titulado a xusticia pola man e dado á luz no seu libro follas novas por certo que s....e desenvolve o proceso de descomposición do sistema-mundo as condicións de intervención de capital e
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu....malo de estado español e o proceso de descomposición do poder constitucional e desembocar nestas ase


## lr 0.01 RMSprot batch 500

In [ ]:
add_log("\n\nlr 0.01 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr01.hdf5')



In [11]:
model=create_gru_model(chars)
print_predicctions(model,'../model_weights/best_beiras_gru_textdata_weights-lr01.hdf5',
                   chars_to_indices,indices_to_chars,text_clean,window_size)

pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran....                                                                                                    
poema de rosalía titulado a xusticia pola man e dado á luz no seu libro follas novas por certo que s....                                                                                                    
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu....                                                                                                    


## lr 0.1 RMSprot batch 500

In [ ]:
add_log("\n\nlr 0.1 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.1, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('../model_weights/best_beiras_gru_textdata_weights-lr1.hdf5')


In [12]:
model=create_gru_model(chars)
print_predicctions(model,'../model_weights/best_beiras_gru_textdata_weights-lr1.hdf5',
                   chars_to_indices,indices_to_chars,text_clean,window_size)

pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran....aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
poema de rosalía titulado a xusticia pola man e dado á luz no seu libro follas novas por certo que s....aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu....aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


## lr 0.001 Adam bacch 500

In [ ]:
add_log("\n\nlr 0.001 Adam bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.Adam(lr=0.001)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('../model_weights/best_beiras_gru_textdata_weights-adam-lr001.hdf5')


/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
 584000/1174280 [=============>................] - ETA: 614s - loss: 2.0793 

In [14]:
model=create_gru_model(chars)
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-adam-lr001.hdf5',
                   chars_to_indices,indices_to_chars,text_clean,window_size)

pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran.... o seu proprio jean-paul sartre e de estado de construir a sua actividade e a democracia para o proc
poema de rosalía titulado a xusticia pola man e dado á luz no seu libro follas novas por certo que s....e acaso non se está a ser de transformación de cartón de cada caso da consulta en toda a consulta en
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu....mental do consello nacional da consulta en toda a consulta en toda a consulta en toda a consulta en 


## lr 0.001 RSProp bacch 3
Too slow

In [ ]:
add_log("\n\nlr 0.001 RSProp bacch 32")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=32, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('../model_weights/best_beiras_gru_textdata_weights-batch32.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-batch32.hdf5')

/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
  24256/1174280 [..............................] - ETA: 11939s - loss: 2.4379